In [2]:
from pyspark.sql import DataFrameReader
from pyspark.sql import SparkSession
from pyspark.ml.feature import IndexToString, Normalizer, StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from helpers.helper_functions import translate_to_file_string
from pyspark.sql.functions import col,lit,to_date


# for pretty printing
def printDf(sprkDF): 
    newdf = sprkDF.toPandas()
    from IPython.display import display, HTML
    return HTML(newdf.to_html())

In [3]:
inputFile = translate_to_file_string("./data/RKI_COVID19_20210529.csv")

### Create Spark Session

In [4]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("RKICOVID19UNDERSTANDING")
       .getOrCreate())
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ",") \
       .csv(inputFile)   
print(df.printSchema())

root
 |-- ObjectId: integer (nullable = true)
 |-- IdBundesland: integer (nullable = true)
 |-- Bundesland: string (nullable = true)
 |-- Landkreis: string (nullable = true)
 |-- Altersgruppe: string (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- AnzahlFall: integer (nullable = true)
 |-- AnzahlTodesfall: integer (nullable = true)
 |-- Meldedatum: string (nullable = true)
 |-- IdLandkreis: integer (nullable = true)
 |-- Datenstand: string (nullable = true)
 |-- NeuerFall: integer (nullable = true)
 |-- NeuerTodesfall: integer (nullable = true)
 |-- Refdatum: string (nullable = true)
 |-- NeuGenesen: integer (nullable = true)
 |-- AnzahlGenesen: integer (nullable = true)
 |-- IstErkrankungsbeginn: integer (nullable = true)
 |-- Altersgruppe2: string (nullable = true)

None


### Show First Row

In [5]:
df.show(1)

+--------+------------+------------------+------------+------------+----------+----------+---------------+--------------------+-----------+--------------------+---------+--------------+--------------------+----------+-------------+--------------------+-----------------+
|ObjectId|IdBundesland|        Bundesland|   Landkreis|Altersgruppe|Geschlecht|AnzahlFall|AnzahlTodesfall|          Meldedatum|IdLandkreis|          Datenstand|NeuerFall|NeuerTodesfall|            Refdatum|NeuGenesen|AnzahlGenesen|IstErkrankungsbeginn|    Altersgruppe2|
+--------+------------+------------------+------------+------------+----------+----------+---------------+--------------------+-----------+--------------------+---------+--------------+--------------------+----------+-------------+--------------------+-----------------+
|       1|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|         3|              0|2021/03/19 00:00:...|       1001|29.05.2021, 00:00...|        0|            -9|20

In [6]:
### Anzahl der Datensätze
print(df.count())

2003106


In [7]:
### Alle betroffenen Bundesländer
df.groupBy("Bundesland").count().show()

+--------------------+------+
|          Bundesland| count|
+--------------------+------+
|      Sachsen-Anhalt| 54705|
|       Niedersachsen|156959|
|         Brandenburg| 70900|
|              Berlin| 97152|
|              Bayern|362635|
|             Sachsen|118722|
|             Hamburg| 26470|
|              Bremen| 11812|
| Nordrhein-Westfalen|393861|
|           Thüringen| 83128|
|              Hessen|151970|
|   Baden-Württemberg|282334|
|Mecklenburg-Vorpo...| 28366|
|            Saarland| 22127|
|  Schleswig-Holstein| 47239|
|     Rheinland-Pfalz| 94726|
+--------------------+------+



In [8]:
### Anzahl der Datensätze nach Bundesland
# raus machen denle, ich braucht zu lange
df.describe().show()

+-------+-----------------+-----------------+-----------------+-------------------+------------+----------+------------------+-------------------+--------------------+-----------------+--------------------+--------------------+------------------+--------------------+-------------------+------------------+--------------------+-----------------+
|summary|         ObjectId|     IdBundesland|       Bundesland|          Landkreis|Altersgruppe|Geschlecht|        AnzahlFall|    AnzahlTodesfall|          Meldedatum|      IdLandkreis|          Datenstand|           NeuerFall|    NeuerTodesfall|            Refdatum|         NeuGenesen|     AnzahlGenesen|IstErkrankungsbeginn|    Altersgruppe2|
+-------+-----------------+-----------------+-----------------+-------------------+------------+----------+------------------+-------------------+--------------------+-----------------+--------------------+--------------------+------------------+--------------------+-------------------+------------------+--

In [9]:
# Anzahl der Fälle nach Bundesland
df.filter(df['NeuerFall'] >  0).groupBy("Bundesland").count().show()


+--------------------+-----+
|          Bundesland|count|
+--------------------+-----+
|      Sachsen-Anhalt|  122|
|       Niedersachsen|  370|
|         Brandenburg|   86|
|              Berlin|  308|
|              Bayern|  608|
|             Sachsen|  274|
|             Hamburg|   33|
|              Bremen|   16|
| Nordrhein-Westfalen|  766|
|           Thüringen|  150|
|              Hessen|  319|
|   Baden-Württemberg|  614|
|Mecklenburg-Vorpo...|   40|
|            Saarland|   51|
|  Schleswig-Holstein|   94|
|     Rheinland-Pfalz|  216|
+--------------------+-----+



In [10]:
# Anzahl der Gesamtfälle
df.groupBy().sum("AnzahlFall").show()

+---------------+
|sum(AnzahlFall)|
+---------------+
|        3674704|
+---------------+



In [11]:
df.filter(df['NeuerFall'] >  0).count()

4067

## Fälle insgesamt (abzüglich Korrekturen - Feld NeuerFall = -1)

In [12]:
df.groupBy().sum("AnzahlFall").show()

+---------------+
|sum(AnzahlFall)|
+---------------+
|        3674704|
+---------------+



## Fälle insgesamt (ohne Korrekturen - siehe https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html)

In [13]:
df.filter(df['NeuerFall'] >  -1).groupBy().sum("AnzahlFall").show()

+---------------+
|sum(AnzahlFall)|
+---------------+
|        3675296|
+---------------+



## Anzahl neue Fälle (heute) (mit Korrekturen - siehe https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html)

In [14]:
df.filter(df['NeuerFall']  != 0).groupBy().sum("AnzahlFall").show()

+---------------+
|sum(AnzahlFall)|
+---------------+
|           5426|
+---------------+



## Fälle kategorsiert nach NeuerFall-Typ (1 = Neuer Fall, 0 = Bekannter Fall, -1 Korrektur)

In [15]:
df.groupBy("NeuerFall").sum("AnzahlFall").show()

+---------+---------------+
|NeuerFall|sum(AnzahlFall)|
+---------+---------------+
|       -1|           -592|
|        1|           6018|
|        0|        3669278|
+---------+---------------+



# Anzahl Todesfall

In [16]:
df.groupBy("NeuerTodesfall").sum("AnzahlTodesfall").show()

+--------------+--------------------+
|NeuerTodesfall|sum(AnzahlTodesfall)|
+--------------+--------------------+
|            -1|                 -10|
|             1|                 173|
|            -9|                   0|
|             0|               88177|
+--------------+--------------------+



In [17]:
# Anzahl Todesfall nach Geschlecht
df.groupBy("NeuerTodesfall","Geschlecht").sum("AnzahlTodesfall").sort(col("NeuerTodesfall").desc()).show()

+--------------+----------+--------------------+
|NeuerTodesfall|Geschlecht|sum(AnzahlTodesfall)|
+--------------+----------+--------------------+
|             1|         W|                  71|
|             1|         M|                 102|
|             0|         M|               46125|
|             0| unbekannt|                 207|
|             0|         W|               41845|
|            -1|         M|                  -7|
|            -1|         W|                  -3|
|            -9|         M|                   0|
|            -9| unbekannt|                   0|
|            -9|         W|                   0|
+--------------+----------+--------------------+



# Anzahl Genesen

In [18]:
df.groupBy("NeuGenesen").sum("AnzahlGenesen").show()

+----------+------------------+
|NeuGenesen|sum(AnzahlGenesen)|
+----------+------------------+
|        -1|              -691|
|         1|             10829|
|        -9|                 0|
|         0|           3461001|
+----------+------------------+



## Prüfen ob Zellen Infos zu Todefällen, Genesungen und Neufällen enthält

In [19]:
df.filter(((df["AnzahlFall"] > 0) & (df["AnzahlTodesfall"] > 0)) | ((df["AnzahlGenesen"] > 0) & (df["AnzahlTodesfall"] > 0)) | ((df["AnzahlFall"] > 0) & (df["AnzahlGenesen"] > 0))).count()

1941511

## Prüfen ob es Einträge gibt, in denen alle drei gefüllt sind

In [20]:
df.filter((df["AnzahlFall"] > 0) & (df["AnzahlTodesfall"] > 0) & (df["AnzahlGenesen"] > 0)).count()

0

## Prüfen ob es Records gibt, in den Anzahl Genesen und Anzahl Todesfall gefüllt sind

In [21]:
df.filter((df["AnzahlGenesen"] > 0) & (df["AnzahlTodesfall"] > 0)).count()

0

## Prüfen wie viele Records gibt, in den Anzahl Fall und Anzahl Todesfall gefüllt sind

In [22]:
df.filter((df["AnzahlFall"] > 0) & (df["AnzahlTodesfall"] > 0)).count()

73805

## Prüfen wie viele Records gibt, in den Anzahl Fall und Anzahl Genesen gefüllt sind

In [23]:
df.filter((df["AnzahlFall"] > 0) & (df["AnzahlGenesen"] > 0)).count()

1867706

## Prüfen ob es Einträge gibt bei denen Anzahl Fall und Anzahl Todesfall nicht den selben Wert haben

In [24]:
df.filter((df["AnzahlFall"] > 0) & (df["AnzahlTodesfall"] > 0) & (df["AnzahlFall"] != df["AnzahlTodesfall"])).count()

0

## Gegenprobe

In [25]:
df.filter((df["AnzahlFall"] > 0) & (df["AnzahlTodesfall"] > 0) & (df["AnzahlFall"] == df["AnzahlTodesfall"])).count()

73805

## Prüfen ob es Einträge gibt bei denen Anzahl Genesen und Anzahl Fall nicht den selben Wert haben

In [26]:
df.filter((df["AnzahlFall"] > 0) & (df["AnzahlGenesen"] > 0) & (df["AnzahlFall"] != df["AnzahlGenesen"])).count()

0

## Gegenprobe

In [27]:
df.filter((df["AnzahlFall"] > 0) & (df["AnzahlGenesen"] > 0) & (df["AnzahlFall"] == df["AnzahlGenesen"])).count()

1867706

## Prüfen ob es Fälle gibt bei denen Anzal Fall 0 ist und Anzahl Geenesen bzw- Todesfall größer 0 sind

In [28]:
df.filter((df["AnzahlFall"] == 0) & ((df["AnzahlGenesen"] > 0) | (df["AnzahlTodesfall"]  > 0))).count()

0

In [29]:
## Prüfen ob es Einträge gibt bei denen Anzahl Genesen und Anzahl Todesfall nicht den selben Wert haben

In [30]:
#Spalte einfügen um Datumstring in DateType umzuwandeln
df.withColumn("DateTime",to_date(col("Refdatum"), "yyyy/MM/dd HH:mm:ssx")).show(1)


#df.filter(df['NeuerFall'] == 1).filter(to_date(df.Refdatum, "dd/MM/yyyy") == lit("2021-05-29")).groupBy("NeuerFall").sum("AnzahlFall").show()


+--------+------------+------------------+------------+------------+----------+----------+---------------+--------------------+-----------+--------------------+---------+--------------+--------------------+----------+-------------+--------------------+-----------------+----------+
|ObjectId|IdBundesland|        Bundesland|   Landkreis|Altersgruppe|Geschlecht|AnzahlFall|AnzahlTodesfall|          Meldedatum|IdLandkreis|          Datenstand|NeuerFall|NeuerTodesfall|            Refdatum|NeuGenesen|AnzahlGenesen|IstErkrankungsbeginn|    Altersgruppe2|  DateTime|
+--------+------------+------------------+------------+------------+----------+----------+---------------+--------------------+-----------+--------------------+---------+--------------+--------------------+----------+-------------+--------------------+-----------------+----------+
|       1|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|         3|              0|2021/03/19 00:00:...|       1001|29.05.2021, 00:

## Untersuchung Altersgruppe

In [32]:
#Werte Altersgruppe
df.groupBy("Altersgruppe").count().show()

+------------+------+
|Altersgruppe| count|
+------------+------+
|     A35-A59|678082|
|     A00-A04| 75024|
|        A80+|171094|
|     A05-A14|169552|
|     A15-A34|557468|
|     A60-A79|349457|
|   unbekannt|  2429|
+------------+------+



## Untersuchung Ausprägungen Geschlecht

In [35]:
df.groupBy("Geschlecht").count().show()

+----------+-------+
|Geschlecht|  count|
+----------+-------+
|         M| 969988|
|         W|1013266|
| unbekannt|  19852|
+----------+-------+



In [33]:
## TODO:
#-> Geschlecht Range
#--> Datumsrange
#-_> Zeigen das Altersgruppe 2 leer ist
#--> Visualisierung
#--> Feld ist erkrankungsbeginn
#--> Wie viele Geshclecht unbekannt

#--> Aufnehmen von asugewählten Kominationen:
#    + wenn alle Korrekturen herausgefilter sind, Fälle anschauen neu genesen -1 und neuerTodesfall -1
#    + Schauen wie viel neuer Fall
#    +


SyntaxError: invalid syntax (<ipython-input-33-4baba338acc6>, line 2)

In [ ]:
# DateFormart
dfNeu = df.withColumn("DateTime",to_date(col("Refdatum"), "yyyy/MM/dd HH:mm:ssx"))



In [ ]:
dfNeu.sort(col("DateTime").desc()).show()

In [ ]:
df.filter(df['NeuerFall'] >  0).groupBy().sum("AnzahlFall").show()